# Project Run

## PIP Installs

In [18]:
# PIPs
# !pip install pydub
# !pip install timm==0.4.5
# !pip install audiomentations
# !pip install torch_audiomentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Drive Mount

In [19]:
# mount gdrive

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Imports

In [20]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import pickle
import json
# 
from pathlib import Path

sys.path.append('/content/gdrive/My Drive/DeepLearning/project/')
sys.path.append('/content/gdrive/My Drive/DeepLearning/project/SSAST/src')
!cd '/content/gdrive/My Drive/DeepLearning/project/SSAST/src'

# cd gdrive/My Drive/DeepLearning/project/
import wav_processing as wp
import project_utils
import birds_run
import birds_ssast_data_setting as birds_ds
from data_validator import Validator


## File Path

In [21]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import pickle
import json

from pathlib import Path

sys.path.append('/content/gdrive/My Drive/DeepLearning/project/')
sys.path.append('/content/gdrive/My Drive/DeepLearning/project/SSAST/src')
!cd '/content/gdrive/My Drive/DeepLearning/project/SSAST/src'

# cd gdrive/My Drive/DeepLearning/project/
import wav_processing as wp
import project_utils
import birds_run
import birds_ssast_data_setting as birds_ds
from data_validator import Validator

import torch

## Copy from Google Drive to Local Colab

In [22]:
!mkdir /content/labeled_data

mkdir: cannot create directory ‘/content/labeled_data’: File exists


In [24]:
!cp -r /content/gdrive/MyDrive/DeepLearning/project/labeled_data/ /content/

In [ ]:
# import os
# os.listdir('/content/labeled_data/')

['birdcalls_validate_data_low_count_filter_100_u.json',
 'part_lists.json',
 'part_lists.pkl',
 'birdcalls_train_data_low_count_filter_10.json',
 'birdcalls_test_data.json',
 'birdcalls_test_data_low_count_filter_100.json',
 'birdcalls_validate_data_low_count_filter_10.json',
 'train',
 'birdcalls_train_data_low_count_filter_100_u.json',
 'birdcalls_test_data_low_count_filter_100_u.json',
 'birdcalls_test_data_low_count_filter_10.json',
 'test',
 'birdcalls_class_labels_indices.csv',
 'validate',
 'label_mask_low_count_filter_10.json',
 'birdcalls_train_data_low_count_filter_100.json',
 'label_mask_low_count_filter_100.json',
 'birdcalls_validate_data_low_count_filter_100.json',
 'birdcalls_train_data.json',
 'birdcalls_validate_data.json']

## Update Jsons to Local Colab Drive

In [25]:
path_replace={
    'from': '/content/gdrive/My Drive/DeepLearning/project/labeled_data/',
    'to': '/content/labeled_data/'
}

source_path = '/content/gdrive/My Drive/DeepLearning/project/labeled_data/'
json_file_list = [
    source_path + 'birdcalls_train_data_low_count_filter_1000.json',
    source_path + 'birdcalls_validate_data_low_count_filter_1000.json',
    source_path + 'birdcalls_test_data_low_count_filter_1000.json'
]

wp.update_dataset_jsons(json_file_list, path_replace)

In [ ]:
import os
labeld_data_path = '/content/gdrive/My Drive/DeepLearning/project/labeled_data/'
# aug_path = Path(labeld_data_path, 'labeled_aug_data')
# len(os.listdir(aug_path))

### Resaqmpling

In [ ]:
from wav_processing import resampling
base_path = '/content/labeled_data/'
json_train= base_path + 'birdcalls_train_data_low_count_filter_10_u.json'
json_train_resample = base_path + 'birdcalls_train_data_low_count_filter_10_u_resample.json'

resampling(json_src=json_train, json_dst=json_train_resample, replicate_count_threshold=1000, minimum_th=10)

FileNotFoundError: ignored

## SSAST Fine-Tune

In [26]:
# Get number of classes
# base_path = '/content/gdrive/My Drive/DeepLearning/project/labeled_data/'
base_path = '/content/labeled_data/'
label_csv= base_path + 'birdcalls_class_labels_indices.csv'
label_df = pd.read_csv(label_csv)
len(label_df)

78

In [27]:
from wav_processing import gen_pos_weight

# Start from default args
args = birds_run.default_args

# pretrain_model='SSAST-Tiny-Patch-400' #SSAST-Base-Frame-400

args.dataset='birdcalls'
args.dataset_mean=-0.22943932
args.dataset_std=0.93046004
args.target_length=128 #1024 # 128
args.noise=True
# Nominal
# args.data_train='/content/gdrive/My Drive/DeepLearning/project/labeled_data/birdcalls_train_data.json'
# args.data_val='/content/gdrive/My Drive/DeepLearning/project/labeled_data/birdcalls_validate_data.json'
# args.data_eval='/content/gdrive/My Drive/DeepLearning/project/labeled_data/birdcalls_test_data.json'
# Partial
# args.data_train='/content/gdrive/My Drive/DeepLearning/project/labeled_data/birdcalls_train_data_partial_4881.json'
# args.data_val='/content/gdrive/My Drive/DeepLearning/project/labeled_data/birdcalls_validate_data_partial_543.json'
# args.data_eval='/content/gdrive/My Drive/DeepLearning/project/labeled_data/birdcalls_test_data_partial_1359.json'
# Low count filter 10 - Multi label
args.data_train= base_path + 'birdcalls_train_data_low_count_filter_1000_u.json'

# args.data_train= base_path + 'birdcalls_train_data_low_count_filter_10_u_resample.json' 

args.data_val= base_path + 'birdcalls_validate_data_low_count_filter_1000_u.json'
args.data_eval= base_path + 'birdcalls_test_data_low_count_filter_1000_u.json'


# Low count filter 10
# args.data_train= base_path + 'birdcalls_train_data_low_count_filter_10_u.json'
# args.data_val= base_path + 'birdcalls_validate_data_low_count_filter_10_u.json'
# args.data_eval= base_path + 'birdcalls_test_data_low_count_filter_10_u.json'

args.label_csv=label_csv
args.n_class = len(label_df)

args.bal=False
args.lr=3e-4  # 3e-4

# args.freqm=48 # 0
# args.timem=48 # 0
args.mixup=0.6 # 0.6
args.fshape=16
args.tshape=16
args.fstride=16 # 16
args.tstride=20 # 16

args.task='ft_cls'
# args.model_size='small'
args.model_size='base'
# head_lr=1
args.warmup = False

# args.pretrained_mdl_path='/content/gdrive/My Drive/DeepLearning/project/SSAST/SSAST-Tiny-Patch-400.pth'
# args.pretrained_mdl_path='/content/gdrive/My Drive/DeepLearning/project/SSAST/SSAST-Small-Patch-400.pth'
args.pretrained_mdl_path='/content/gdrive/My Drive/DeepLearning/project/SSAST/SSAST-Base-Patch-400.pth'

args.exp_dir='/content/gdrive/My Drive/DeepLearning/project/experiments/11 Report filter Mixing 0_6 w Aug 1000'

args.num_workers=1

args.pos_weight = torch.ones([args.n_class]).tolist()
# args.pos_weight = gen_pos_weight(args.data_train, label_csv).tolist()

args.batch_size=128
args.n_epochs=25

args.aug_en = True


In [28]:
label_mask_path = base_path + 'label_mask_low_count_filter_1000.json'
stats_list = birds_run.run(args, label_mask_path=label_mask_path)

balanced sampler is not used
---------------the train dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.600000
now process birdcalls
use dataset mean -0.229 and std 0.930 to normalize the input.
now use noise augmentation
number of classes is 78
---------------the evaluation dataloader---------------
now using following mask: 0 freq, 0 time
now using mix-up with rate 0.000000
now process birdcalls
use dataset mean -0.229 and std 0.930 to normalize the input.
number of classes is 78
Now train with birdcalls with 6188 training samples, evaluate with 720 samples
now load a SSL pretrained models from /content/gdrive/My Drive/DeepLearning/project/SSAST/SSAST-Base-Patch-400.pth
pretraining patch split stride: frequency=16, time=16
pretraining patch shape: frequency=16, time=16
pretraining patch array dimension: frequency=8, time=64
pretraining number of patches=512
f_dim=8, t_dim=6
p_f_dim=8, p_t_dim=64
fstride=16, tstride=20, input_fdim=128, inp

48it [03:02,  3.80s/it]

start validation
label_mask=tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], device='cuda:0')


target[pred_positive_indexes].sum()=17.0
len(pred_positive_indexes)=15
(output[gt_positive_indexes] > 0.5).sum()=26203
len(gt_positive_indexes)=810
acc: 0.000000
birds_acc: 0.985737
birds_precision: 1.133333
birds_recall: 32.349384
train_loss: 0.490336
valid_loss: 0.751184
*** Save models
normal learning rate scheduler step
Epoch-1 lr: 0.0003
Epoch-1 lr: 0.0003
epoch 1 training time: 209.311
---------------
2022-10-17 20:18:32.402019
current #epochs=2, #steps=48


48it [01:56,  2.42s/it]

...

start validation
label_mask=tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], device='cuda:0')


target[pred_positive_indexes].sum()=888.0
len(pred_positive_indexes)=762
(output[gt_positive_indexes] > 0.5).sum()=30126
len(gt_positive_indexes)=810
acc: 0.568056
birds_acc: 0.989744
birds_precision: 1.165354
birds_recall: 37.192593
train_loss: 0.237966
valid_loss: 0.691497
*** Save models
normal learning rate scheduler step
Epoch-24 lr: 1.1627859325354297e-05
Epoch-24 lr: 1.1627859325354297e-05
epoch 24 training time: 112.509
---------------
2022-10-17 21:03:21.585780
current #epochs=25, #steps=1152


48it [01:51,  2.31s/it]

start validation
label_mask=tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], device='cuda:0')


target[pred_positive_indexes].sum()=880.0
len(pred_positive_indexes)=753
(output[gt_positive_indexes] > 0.5).sum()=30471
len(gt_positive_indexes)=810
acc: 0.572222
birds_acc: 0.989939
birds_precision: 1.168659
birds_recall: 37.618519
train_loss: 0.230139
valid_loss: 0.690217
*** Save models
normal learning rate scheduler step
Epoch-25 lr: 9.883680426551152e-06
Epoch-25 lr: 9.883680426551152e-06
epoch 25 training time: 124.083
label_mask=tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], device='cuda:0')
target[pred_positive_indexes].sum()=885.0
len(pred_positive_indexes)=761
(output[gt_positive_indexes] > 0.5).sum()=29674
len(gt_positive_indexes)=810
---------------evaluate on the validation set---------------
Accuracy: 0.583333
Birds Accuracy: 0.990153
AUC: 0.066819
------

## GPU Status

In [ ]:
!nvidia-smi

Mon Sep  5 08:44:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
train_json = args.data_train
train_json

'/content/labeled_data/birdcalls_train_data_low_count_filter_100_u.json'

In [ ]:
import json
with open(train_json, 'r') as f:
  a = json.load(f)['data']

In [ ]:
a[0]

{'wav': '/content/labeled_data/train/10_20200405_064600_0.WAV',
 'labels': '/m/birdc37'}